# Visualization of Torsiondrive 1D scan result as scatter plot and interactively showing molecular structures.


#### Known issues: this notebook doesn't work if opened in jupyterlab
### Install required packages for running this script (Run in terminal)
conda install -c conda-forge matplotlib plotly nglview ipywidgets
jupyter-nbextension enable nglview --py --sys-prefix

### Specify path to scan.xyz (produced by torsiondrive 1D scan)

In [1]:
import urllib.request
# Use an example file downloaded from the torsiondrive_examples repo
url = "https://raw.githubusercontent.com/lpwgroup/torsiondrive_examples/master/examples/hooh-1d/psi4/run_local/geomeTRIC/scan.xyz"
urllib.request.urlretrieve(url, 'scan.xyz')
# This path can be replaced by your own file path
scanxyz = "scan.xyz"

### Parse the scan.xyz file

In [2]:
import numpy as np
from torsiondrive.tools import plot_1d

grid_data = plot_1d.read_scan_xyz(scanxyz)
# put grid data into 1-d arrays
x_data, y_data = [], []
for gid, energy in grid_data.items():
    x_data.append(gid[0])
    y_data.append(energy)
# convert abs energies in Hartree to relative energies in kcal/mol
y_data = np.array(y_data)
y_data = (y_data - y_data.min())* 627.509

### Customized nglview class to visualize molecular structures

In [3]:
import nglview
from geometric.molecule import Molecule

class MyStructureTrajectory(nglview.Structure, nglview.Trajectory):
    """ 
    Custom nglview.Structure and nglview.Trajectory subclass
    For loading molecule files using geomeTRIC.molecule.Molecule

    Reference
    ---------
    http://nglviewer.org/nglview/latest/interface_classes.html
    """
    ext = "pdb"  # or gro, cif, mol2, sdf
    params = {}  # loading options passed to NGL
    id = '123121'
    def __init__(self, input_file, *args, **kwargs):
        mol = Molecule(input_file)
        # fill in required fields for PDB format
        if not mol.Data.get('resname'):
            mol.Data['resname'] = ['RES'] * mol.na
        if not mol.Data.get('resid'):
            mol.Data['resid'] = [1] * mol.na
        self.mol = mol
    
    def get_structure_string(self):
        return '\n'.join(self.mol.write_pdb(None))

    def get_coordinates(self, index):
        # return 2D numpy array, shape=(n_atoms, 3)
        return self.mol.xyzs[index]
    
    @property
    def n_frames(self):
        # return total frames
        return len(self.mol)

### Generate 1-D torsion angle vs energy plot

In [4]:
import plotly
import plotly.graph_objs as go

# The nglview widget for molecule
molview = nglview.NGLWidget(MyStructureTrajectory(scanxyz))

# plotly Scatter plot
scatter = go.Scatter(x=x_data, y=y_data, mode='lines+markers')
marker_color_list = ['#1f77b4'] * len(x_data)
scatter.marker.color = marker_color_list
marker_size_list = [10] * len(x_data)
scatter.marker.size = marker_size_list
# click function
frame_idx = {gid: idx for idx, gid in enumerate(grid_data)}
def show_structure_for_click(trace, points, selector):
    point_idx = points.point_inds[0]
    # update color
    color_list_copy = marker_color_list.copy()
    color_list_copy[point_idx] = '#2ca02c'
    scatter.marker.color = color_list_copy
    # update size
    marker_size_list_copy = marker_size_list.copy()
    marker_size_list_copy[point_idx] *= 2
    scatter.marker.size = marker_size_list_copy
    # select frame
    gid = (points.xs[0],)
    molview.frame = frame_idx[gid]
fig = go.FigureWidget([scatter])
fig.layout = go.Layout(xaxis=dict(tickvals=x_data, tickangle=30, title='Torion Angle (degree)'), yaxis=dict(title='Relative Energy (kcal/mol)'), hovermode='closest')
scatter = fig.data[0]
scatter.on_click(show_structure_for_click)

# display the plot and structure side by side
import ipywidgets
import IPython.display as display

topdownbox = ipywidgets.VBox([fig, molview])
## Finally, show.
display.display(topdownbox)

    'data': [{'marker': {'color': [#1f77b4, #1f77b4, #1f77b4, #1f77b4, #1f77b4,
…

## Alternatively, you can generate a pdf file of the energy plot

In [5]:
plot_filename = 'torsiondrive-1d-plot.pdf'
plot_1d.plot_1d_curve(grid_data, plot_filename)